## 构建卷积神经网络

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline

## 读取数据

In [9]:
# 定义超参数
input_size = 28 # 图像尺寸大小为28*28
num_classes = 10
num_epochs = 3
batch_size = 64

# 训练集
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# 构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

## 构建卷积网络
* 可以把一组卷积层、relu和池化写成一个模块

In [12]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()     # 完成父类的初始化构造
        self.conv1 = nn.Sequential(     # 输入大小（1，28，28）
            nn.Conv2d(in_channels=1,    # 输入图片的通道数量，1就是灰度图
                      out_channels=16,  # 输出的特征图的通道数量
                      kernel_size=5,    # 卷积核大小，与特征图的大小成反比
                      stride=1, # 卷积核移动的步长
                      padding=2 # 边缘填充
                      ),        # 输出的特征图的大小是（16， 28，28）
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  # 进行池化操作，在2*2的区域 中选出最大的数值
        )   # 第一个模块输出的大小是（16，14，14）通道数，高度，宽度
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2 ), # 输出的特征图的大小是（32，14，14）
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), 
        )   # 第二个模块输出的大小是（32，7，7）通道数，高度，宽度
        self.out = nn.Linear(32 * 7 * 7, num_classes)   # 全连接层，输入是拉伸后的一维向量
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)   # 输入是希望得到的新的尺寸(batch_size, 32*7*7),原x尺寸为(batch_size, channels, height, width)，-1是自动计算第二个维度的
        x = self.out(x)
        return x
        

## 准确率评估

In [17]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data).view_as(pred).sum()
    return rights, len(labels)

## 模型训练

In [18]:
net = CNN() # 实例化网络
criterion = nn.CrossEntropyLoss()   # 损失函数，交叉熵函数用于分类任务
optimizer = optim.Adam(net.parameters(), lr=0.001)  # 优化器

for epoch in range(num_epochs):
    train_rights = []   # 保存当前epoch的结果
    
    for batch_idx, (data, target) in enumerate(train_loader):   # 对loader中的每个batch进行循环
        net.train()
        output = net(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            
            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
            
            # 计算准确率
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
            
            print(f"Epoch {epoch+1}/{num_epochs}.. 损失{loss.data} 训练集准确率：{train_r[0].numpy() / train_r[1]}, 测试集准确率：{val_r[0].numpy() / val_r[1]}")
        

Epoch 1/3.. 损失2.3004086017608643 训练集准确率：0.1875, 测试集准确率：0.0958
Epoch 1/3.. 损失0.3133316934108734 训练集准确率：0.7872834158415841, 测试集准确率：0.9283
Epoch 1/3.. 损失0.24484160542488098 训练集准确率：0.857353855721393, 测试集准确率：0.9529
Epoch 1/3.. 损失0.11253324896097183 训练集准确率：0.8892753322259136, 测试集准确率：0.9648
Epoch 1/3.. 损失0.053633373230695724 训练集准确率：0.90776963840399, 测试集准确率：0.9694
Epoch 1/3.. 损失0.12236596643924713 训练集准确率：0.9192864271457086, 测试集准确率：0.9731
Epoch 1/3.. 损失0.05259229242801666 训练集准确率：0.9281146006655574, 测试集准确率：0.9799
Epoch 1/3.. 损失0.1264343112707138 训练集准确率：0.9349366975748931, 测试集准确率：0.9783
Epoch 1/3.. 损失0.05717533454298973 训练集准确率：0.9398213171036205, 测试集准确率：0.9758
Epoch 1/3.. 损失0.10066297650337219 训练集准确率：0.9437950887902331, 测试集准确率：0.9801
Epoch 2/3.. 损失0.01573813520371914 训练集准确率：1.0, 测试集准确率：0.9767
Epoch 2/3.. 损失0.13630877435207367 训练集准确率：0.9789603960396039, 测试集准确率：0.981
Epoch 2/3.. 损失0.160904660820961 训练集准确率：0.9796330845771144, 测试集准确率：0.9846
Epoch 2/3.. 损失0.016554001718759537 训练集准确率：0.9801702657807309